### Hackthon 6, Team 3

In [58]:
# imports

import os
import base64
import joblib
import pandas as pd
import numpy as np
import category_encoders
import json
import joblib
import pickle
import math
import requests
from copy import deepcopy
import seaborn as sns
from uuid import uuid4

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_score
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [59]:
def load_data():
    df = pd.read_csv(os.path.join("data", "train.csv"))
    return df

df = load_data()
df.head()

,Unnamed: 0,admission_id,patient_id,race,gender,age,weight,admission_type_code,discharge_disposition_code,admission_source_code,...,blood_type,hemoglobin_level,blood_transfusion,max_glu_serum,A1Cresult,diuretics,insulin,change,diabetesMed,readmitted
0,62479,62479,46826964,White,Male,[80-90),?,3.0,1.0,1,...,A-,16.0,False,NONE,None,No,No,No,No,No
1,4543,4543,57647952,Caucasian,Male,[40-50),?,6.0,25.0,17,...,O+,14.2,False,>200,None,No,Yes,No,Yes,No
2,61307,61307,73513008,European,Female,[30-40),?,2.0,1.0,7,...,O+,12.7,False,None,None,No,Yes,No,Yes,No
3,43014,43014,204181182,Caucasian,Female,[40-50),?,1.0,6.0,7,...,O+,12.3,False,None,None,No,Yes,Ch,Yes,Yes
4,64051,64051,47947464,AfricanAmerican,Male,[70-80),?,1.0,11.0,5,...,O+,13.5,False,None,None,No,Yes,Ch,Yes,No


In [61]:
df.dtypes

Unnamed: 0                       int64
admission_id                     int64
patient_id                       int64
race                            object
gender                          object
age                             object
weight                          object
admission_type_code            float64
discharge_disposition_code     float64
admission_source_code            int64
time_in_hospital                 int64
payer_code                      object
medical_specialty               object
has_prosthesis                    bool
complete_vaccination_status     object
num_lab_procedures             float64
num_procedures                   int64
num_medications                float64
number_outpatient                int64
number_emergency                 int64
number_inpatient                 int64
diag_1                          object
diag_2                          object
diag_3                          object
number_diagnoses                 int64
blood_type               

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   10000 non-null  int64  
 1   admission_id                 10000 non-null  int64  
 2   patient_id                   10000 non-null  int64  
 3   race                         10000 non-null  object 
 4   gender                       10000 non-null  object 
 5   age                          9724 non-null   object 
 6   weight                       9820 non-null   object 
 7   admission_type_code          9856 non-null   float64
 8   discharge_disposition_code   9930 non-null   float64
 9   admission_source_code        10000 non-null  int64  
 10  time_in_hospital             10000 non-null  int64  
 11  payer_code                   10000 non-null  object 
 12  medical_specialty            10000 non-null  object 
 13  has_prosthesis   

In [63]:
# check numeric variables

df.describe()

,Unnamed: 0,admission_id,patient_id,admission_type_code,discharge_disposition_code,admission_source_code,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,hemoglobin_level
count,10000.000000,10000.000000,1.000000e+04,9856.000000,9930.000000,10000.000000,10000.000000,9800.000000,10000.000000,9681.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,40697.854000,40697.854000,1.082937e+08,2.015422,3.751762,5.748500,4.340900,42.788776,1.313200,16.032848,0.37870,0.183700,0.623100,7.409000,14.179000
std,23510.882143,23510.882143,7.735092e+07,1.425137,5.318510,4.094524,2.935839,19.666400,1.683327,8.084788,1.32373,0.818304,1.250281,1.922469,1.068013
min,0.000000,0.000000,2.232000e+03,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.00000,0.000000,0.000000,1.000000,10.900000
25%,20367.250000,20367.250000,4.687067e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.00000,0.000000,0.000000,6.000000,13.400000
50%,40715.500000,40715.500000,9.073632e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.00000,0.000000,0.000000,8.000000,14.100000
75%,61071.000000,61071.000000,1.734926e+08,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.00000,0.000000,1.000000,9.000000,15.000000
max,81405.000000,81405.000000,3.766467e+08,8.000000,28.000000,22.000000,14.000000,132.000000,6.000000,70.000000,35.00000,22.000000,18.000000,16.000000,18.100000


In [64]:
# check unique values

for col in df.columns:
    if col not in ['Unnamed: 0', 'admission_id', 'patient_id', 'age', 'weight', 'diag_2']:
        print("{}: {} \n".format(col, sorted(df[col].unique())))

race: ['?', 'AFRICANAMERICAN', 'African American', 'AfricanAmerican', 'Afro American', 'Asian', 'Black', 'Caucasian', 'EURO', 'European', 'Hispanic', 'Latino', 'Other', 'WHITE', 'White'] 

gender: ['Female', 'Male', 'Unknown/Invalid'] 

admission_type_code: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, nan] 

discharge_disposition_code: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 11.0, 13.0, 14.0, 15.0, 16.0, 18.0, 23.0, 24.0, 25.0, nan, 22.0, 28.0] 

admission_source_code: [1, 2, 3, 4, 5, 6, 7, 8, 9, 17, 20, 22] 

time_in_hospital: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] 

payer_code: ['?', 'BC', 'CH', 'CM', 'CP', 'DM', 'HM', 'MC', 'MD', 'MP', 'OG', 'OT', 'PO', 'SI', 'SP', 'UN', 'WC'] 

medical_specialty: ['?', 'AllergyandImmunology', 'Anesthesiology', 'Anesthesiology-Pediatric', 'Cardiology', 'Cardiology-Pediatric', 'DCPTEAM', 'Emergency/Trauma', 'Endocrinology', 'Endocrinology-Metabolism', 'Family/GeneralPractice', 'Gastroenterology', 'Gynecology', 'Hematology', 'Hematology/Onc

In [65]:
df['age'].unique()

array(['[80-90)', '[40-50)', '[30-40)', '[70-80)', '[90-100)', '[50-60)',
       '[60-70)', '[20-30)', nan, '[10-20)', '[0-10)'], dtype=object)

In [66]:
df['weight'].unique()

array(['?', nan, '[50-75)', '[25-50)', '[75-100)', '[125-150)',
       '[100-125)', '[0-25)', '[150-175)', '[175-200)'], dtype=object)

In [67]:
df['diag_2'].unique()

array(['788', '425', '250', 'V85', '276', '250.8', '998', '250.02', '584',
       '496', '250.01', '716', '250.1', '493', '280', '112', '197', '707',
       '401', '403', '428', '715', '?', '571', '995', '780', '414', '359',
       '486', '599', '287', '447', '434', '345', '919', '424', '781',
       '214', '305', '518', '300', '511', '585', '427', '295', '553',
       '285', 'E849', '705', '429', '70', '728', '201', '568', nan, '600',
       '733', '512', '411', '153', '530', '250.11', '41', 'E885', '242',
       '244', '997', '574', '250.51', '342', '785', '682', '786', '648',
       'V10', '799', '250.03', '337', '250.43', '413', '303', '250.82',
       '710', '533', '250.6', '578', '727', '492', '807', '410', '560',
       '284', '278', '198', '412', '564', 'V43', '294', '577', '293',
       'E932', '340', '296', 'V45', '595', '491', '996', '162', '402',
       '535', '151', '331', '179', '626', 'V42', '250.4', '218', '784',
       '731', '572', '618', '920', '438', '466', '693', '

In [68]:
def replace_values(df: pd.DataFrame, feature: str, replace_dict: dict):
    """
    Receives a df and replaces feature column using a replacement dictionary
    """

    df[feature] = df[feature].replace(replace_dict)
    return df

race_dict = {
    'African American': 'AfricanAmerican', 
    'Black': 'AfricanAmerican', 
    'Afro American': 'AfricanAmerican', 
    'AFRICANAMERICAN': 'AfricanAmerican',
    'White': 'Caucasian',
    'WHITE': 'Caucasian',
    'European': 'Caucasian',
    'EURO': 'Caucasian',
    'Latino': 'Hispanic',
    '?': 'Other'
    }

df = replace_values(df, 'race', race_dict)
df.head()

,Unnamed: 0,admission_id,patient_id,race,gender,age,weight,admission_type_code,discharge_disposition_code,admission_source_code,...,blood_type,hemoglobin_level,blood_transfusion,max_glu_serum,A1Cresult,diuretics,insulin,change,diabetesMed,readmitted
0,62479,62479,46826964,Caucasian,Male,[80-90),?,3.0,1.0,1,...,A-,16.0,False,NONE,None,No,No,No,No,No
1,4543,4543,57647952,Caucasian,Male,[40-50),?,6.0,25.0,17,...,O+,14.2,False,>200,None,No,Yes,No,Yes,No
2,61307,61307,73513008,Caucasian,Female,[30-40),?,2.0,1.0,7,...,O+,12.7,False,None,None,No,Yes,No,Yes,No
3,43014,43014,204181182,Caucasian,Female,[40-50),?,1.0,6.0,7,...,O+,12.3,False,None,None,No,Yes,Ch,Yes,Yes
4,64051,64051,47947464,AfricanAmerican,Male,[70-80),?,1.0,11.0,5,...,O+,13.5,False,None,None,No,Yes,Ch,Yes,No


#### Set the features

In [69]:
[col for col in df.columns if df[col].dtype == 'int64' or df[col].dtype == 'float64' ]

['Unnamed: 0',
 'admission_id',
 'patient_id',
 'admission_type_code',
 'discharge_disposition_code',
 'admission_source_code',
 'time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses',
 'hemoglobin_level']

In [70]:
[col for col in df.columns if df[col].dtype == 'O' or df[col].dtype == 'bool']

['race',
 'gender',
 'age',
 'weight',
 'payer_code',
 'medical_specialty',
 'has_prosthesis',
 'complete_vaccination_status',
 'diag_1',
 'diag_2',
 'diag_3',
 'blood_type',
 'blood_transfusion',
 'max_glu_serum',
 'A1Cresult',
 'diuretics',
 'insulin',
 'change',
 'diabetesMed',
 'readmitted']

In [71]:
[col for col in df.columns if df[col].dtype == 'bool']

['has_prosthesis', 'blood_transfusion']

In [72]:
# set features (target, numerical features and categorical features)

target = 'readmitted'

numerical_features = ['admission_id', 'patient_id', 'admission_type_code', 'admission_source_code', 'discharge_disposition_code',
                      'admission_source_code', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications',
                     'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'hemoglobin_level']

# categorical_features = ['race', 'gender', 'age', 'weight', 'payer_code', 'medical_specialty', 'has_prosthesis', 
#                         'complete_vaccination_status', 'diag_1', 'diag_2', 'diag_3', 'blood_type', 'blood_transfusion', 
#                         'max_glu_serum', 'A1Cresult', 'diuretics', 'insulin', 'change', 'diabetesMed']

categorical_features = ['race', 'age', 'weight', 'payer_code', 'medical_specialty', 
                        'complete_vaccination_status', 'diag_1', 'diag_2', 'diag_3', 'blood_type',  
                        'max_glu_serum', 'A1Cresult', 'diuretics', 'insulin', 'change', 'diabetesMed']

In [73]:
len(df.columns), (len(numerical_features) + len(categorical_features))

# we should drop 'Unnamed: 0'

(35, 31)

#### Convert target feature

In [74]:
# convert target feature to True/False

df[target].value_counts()

# df[target][df[target] == 'No'] = 0
# df[target][df[target] == 'Yes'] = 1
# df[target].astype('int32')

No     8942
Yes    1058
Name: readmitted, dtype: int64

#### train_test split

In [75]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,admission_id,patient_id,race,gender,age,weight,admission_type_code,discharge_disposition_code,admission_source_code,time_in_hospital,...,blood_type,hemoglobin_level,blood_transfusion,max_glu_serum,A1Cresult,diuretics,insulin,change,diabetesMed,readmitted
0,62479,46826964,Caucasian,Male,[80-90),?,3.0,1.0,1,2,...,A-,16.0,False,NONE,None,No,No,No,No,No
1,4543,57647952,Caucasian,Male,[40-50),?,6.0,25.0,17,4,...,O+,14.2,False,>200,None,No,Yes,No,Yes,No
2,61307,73513008,Caucasian,Female,[30-40),?,2.0,1.0,7,2,...,O+,12.7,False,None,None,No,Yes,No,Yes,No
3,43014,204181182,Caucasian,Female,[40-50),?,1.0,6.0,7,3,...,O+,12.3,False,None,None,No,Yes,Ch,Yes,Yes
4,64051,47947464,AfricanAmerican,Male,[70-80),?,1.0,11.0,5,6,...,O+,13.5,False,None,None,No,Yes,Ch,Yes,No


In [76]:
# set X_train, y_train, X_test and y_test

X = df.drop(columns=[target], axis=1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [77]:
len(X_train), len(y_train), len(X_test), len(y_test)

(7000, 7000, 3000, 3000)

#### set the pipeline

In [78]:
# set transformations to numerical features and categorical features

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])


# establish the pipeline

pipeline = make_pipeline(
    preprocessor,
    RandomForestClassifier(max_depth=3, min_samples_leaf=.03, class_weight="balanced", random_state=42, n_jobs=-1),
)


# fit the model

pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['admission_id', 'patient_id',
                                                   'admission_type_code',
                                                   'admission_source_code',
                                                   'discharge_disposition_code',
                                                   'admission_source_code',
                                                   'time_in_hospital',
                                                   'num_lab_procedures',
                            

In [80]:
# compute f1_score

y_pred = pipeline.predict(X_test)

metric = f1_score(y_test, y_pred, average='macro')

metric

0.4757030276887394

In [81]:
# get feature importance

feature_importances = pipeline.named_steps['randomforestclassifier'].feature_importances_

features = categorical_features + numerical_features

feature_imp_dict = dict(zip(features, feature_importances))

sorted_imp = sorted(feature_imp_dict.items(), key=lambda x: x[1], reverse=True)

print("Feature Importances:")
for feature, importance in sorted_imp:
    print(f"{feature}: {importance:.4f}")

Feature Importances:
diuretics: 0.0744
age: 0.0626
blood_type: 0.0616
race: 0.0453
diag_2: 0.0442
insulin: 0.0336
max_glu_serum: 0.0289
diag_3: 0.0266
medical_specialty: 0.0235
weight: 0.0230
diag_1: 0.0224
payer_code: 0.0221
A1Cresult: 0.0213
number_inpatient: 0.0142
diabetesMed: 0.0129
num_procedures: 0.0127
complete_vaccination_status: 0.0124
change: 0.0083
number_emergency: 0.0032
number_outpatient: 0.0014
patient_id: 0.0011
admission_id: 0.0000
admission_type_code: 0.0000
admission_source_code: 0.0000
discharge_disposition_code: 0.0000
time_in_hospital: 0.0000
num_lab_procedures: 0.0000
num_medications: 0.0000
number_diagnoses: 0.0000
hemoglobin_level: 0.0000


#### Serialize the files

In [82]:
TMP_DIR = '/tmp'

In [83]:
# serialize columns names, pipeline and dtypes

# column names

with open(os.path.join(TMP_DIR, 'columns.json'), 'w') as fh:
    json.dump(X_train.columns.tolist(), fh)
    
# pipeline

joblib.dump(pipeline, os.path.join(TMP_DIR, 'pipeline.pickle'))

# dtypes

with open(os.path.join(TMP_DIR, 'dtypes.pickle'), 'wb') as fh:
    pickle.dump(X_train.dtypes, fh)

#### Check descrimination issues

In [48]:
def verify_retrieve_rates(X_test, y_true, y_test, sensitive_column, threshold=0.15):
    """
    Verify retrieval rates for different `sex` instances are 
    not different by more than 10 percentage points
    
    Inputs:
        X_test: features for the test cases
        y_true: true labels for the test cases [0, 1]
        y_test: predictions for the test cases [0, 1]

    Returns: tuple of (success, rate_difference)
        success: True if the condition is satisfied, otherwise False
        rate_difference: difference between each class retrieval rates (as an absolute value) 
        
    """
    # YOUR CODE HERE
       
    issue = X_test[sensitive_column].unique()

    success = False
    rate_score = []
    for g in issue:
        mask = (X_test[sensitive_column] == g)
        rate_score.append(recall_score(y_true[mask], y_pred[mask], pos_label=1, zero_division=0))
    
    rate_difference = abs(rate_score[0] - rate_score[1])
        
    if rate_difference <= threshold:
        success = True
    
    return success, rate_difference

race_check = verify_retrieve_rates(X_test, y_true, y_test, 'race', 0.15)
race_check

In [84]:
# Ekaterina

def verify_gender_rate(X_test, y_test, y_pred):
    X_female, X_male = X_test[X_test["gender"] == "Female"], X_test[X_test["gender"] == "Male"]
    y_true_female, y_true_male = y_test[X_test["gender"] == "Female"], y_test[X_test["gender"] == "Male"]
    y_pred_female, y_pred_male = y_pred[X_test["gender"] == "Female"], y_pred[X_test["gender"] == "Male"]
    
    rate_female = f1_score(y_true_female, y_pred_female, pos_label='Yes') 
    rate_male = f1_score(y_true_male, y_pred_male, pos_label='Yes')
    
    success = True if abs(rate_male - rate_female) <= 0.15 else False
    return (success, abs(rate_male - rate_female))

verify_gender_rate(X_test, y_test, y_pred) 

(True, 0.008620344413295955)

In [85]:
# Ekaterina

def verify_race_rate(X_test, y_test, y_pred):
    race_values = X_test["race"].unique()
    rates = []
    for i in range(len(race_values)):
        for j in range(i+1, len(race_values)):
            race_a, race_b = race_values[i], race_values[j]
    
            X_race_a, X_race_b = X_test[X_test["race"] == race_a], X_test[X_test["race"] == race_b]
            y_true_race_a, y_true_race_b = y_test[X_test["race"] == race_a], y_test[X_test["race"] == race_b]
            y_pred_race_a, y_pred_race_b = y_pred[X_test["race"] == race_a], y_pred[X_test["race"] == race_b]

            rate_race_a = f1_score(y_true_race_a, y_pred_race_a, pos_label='Yes') 
            rate_race_b = f1_score(y_true_race_b, y_pred_race_b, pos_label='Yes')
            rates_difference = abs(rate_race_a - rate_race_b)
            rates.append(abs(rate_race_a - rate_race_b))
    
    success = True if np.max(rates) <= 0.15 else False
    return (success, np.max(rates))

verify_race_rate(X_test, y_test, y_pred) 

(True, 0.035714285714285726)

In [55]:
df['race'].value_counts()

Caucasian           6050
AfricanAmerican     1109
White                755
African American     371
WHITE                363
European             313
?                    209
Black                183
Afro American        160
Hispanic             150
Other                129
EURO                  77
Asian                 64
Latino                48
AFRICANAMERICAN       19
Name: race, dtype: int64